In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

from __future__ import division
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.dpi'] = 2.5 * matplotlib.rcParams['figure.dpi']

import numpy as np
from scipy import special as ss
from collections import OrderedDict

In [ ]:
import george
import george.kernels as kernels
import emcee
from chainconsumer import ChainConsumer
import pyDOE

In [ ]:
from model import dataCompress
from model import model
from model import gproc
from model import utils

# Interpolation of known, analytic distribution

Consider the postulated distribution of binary spin-alignments from Talbot & Thrane (2017), [arXiv:1704.08370](https://arxiv.org/abs/1704.08370). Rather than consider the usual $\chi_\mathrm{eff}$ and $\chi_\mathrm{p}$ parameters, in the authors' model the observed quantities are assumed to be the projection of each binary component's spin projection onto the orbital angular momentum vector, such that: $$z_1 = \hat{L}\cdot\hat{S}_1, \\ z_2 = \hat{L}\cdot\hat{S}_2,$$ where $z_{\{1,2\}} \in [-1,1]$.

Dynamical capture mechanisms in a globular cluster are assumed to produce an isotropic distribution of spin alignments, such that $$p_0(z_1,z_2)=\frac{1}{4}.$$ For field binaries, the evolutionary path of each progenitor star, along with BH natal kicks during supernovae, is assumed to produce a truncated Gaussian distribution of alignments. A hyperparameter controls the degree with which (anti-)alignment is favored. Thus, $$p_1(z_1,z_2) = \frac{2}{\pi}\frac{1}{\sigma_1}\frac{e^{-(z_1-1)^2 / 2\sigma_1^2}}{\mathrm{erf}(\sqrt{2}\sigma_1)} \frac{1}{\sigma_2}\frac{e^{-(z_2-1)^2 / 2\sigma_2^2}}{\mathrm{erf}(\sqrt{2}\sigma_2)}.$$ In this model, $\sigma=0$ produces perfect alignment, while $\sigma=\infty$ tends to the dynamic-capture distribution. 

In [ ]:
## Defining all the functions mentioned above from Talbot & Thrane

def gauss_trunc(x,sigma):
    '''Truncated Gaussian distribution'''
    return ( np.sqrt(2.0 / np.pi) * (1.0 / sigma) * \
            np.exp(-0.5 * (x-1.0)**2.0 / sigma**2.0) / ss.erf(np.sqrt(2.0) * sigma) )

def gauss_full(x,sigma):
    '''Gaussian distribution'''
    return ( np.sqrt(2.0 / np.pi) * (1.0 / sigma) * \
            np.exp(-0.5 * (x-1.0)**2.0 / sigma**2.0))

def gauss_fullprod(x1,x2,sigma1,sigma2):
    '''Product of Gaussian pdfs'''
    return gauss_full(x1,sigma1) * gauss_full(x2,sigma2)

def p0(z1, z2):
    '''Dynamic-capture isotropic distribution of spin alignments.'''
    if z1.shape != z2.shape:
        return 'Error! Your population vectors have unequal lengths'
    else:
        return 0.25 * np.ones(z1.shape)

def p1(z1, z2, sigma1, sigma2):
    '''Field binary distribution of spin alignments'''
    if z1.shape != z2.shape:
        return 'Error! Your population vectors have unequal lengths'
    else:
        return gauss_trunc(z1, sigma1) * gauss_trunc(z2, sigma2)

def p(z1, z2, sigma1, sigma2, xi):
    '''Total population distribution of spin alignments as dynamical/field mixture'''
    return (1.0-xi) * p0(z1, z2) + xi * p1(z1, z2, sigma1, sigma2)

## 1D compression test in sigma space

In [ ]:
class sampleTrans2d(object):
    """Sampling tranformations for sigma"""
    
    def __init__(self):
        pass
    
    def unit2range(self, x):
        return 10.0**(-1.0 + (1.0 - (-1.0)) * x)
    
    def range2unit(self, y):
        return (1.0 + np.log10(y)) / 2.

In [ ]:
# Defining bins in parameter space.

z1edges = np.linspace(-1.0, 1.0, 40)
z2edges = np.linspace(-1.0, 1.0, 40)

z1m = (z1edges[1:] + z1edges[:-1]) / 2.0
z2m = (z2edges[1:] + z2edges[:-1]) / 2.0

# Sort data in 2-d space of z1 and z2 into one long vector

z1v, z2v  = np.meshgrid(z1m, z2m, indexing='ij')
z_sample = np.zeros((np.prod(z1v.shape),2))

z_sample[:,0] = z1v.flatten()
z_sample[:,1] = z2v.flatten() 

In [ ]:
## Also, define the sigma values (really, any parameters of interest) at which simulations are run.
## We will try to emulate the truncated Gaussian distribution.

nsims = 100
ndim = 2

## latin hypercube sampling
## consider on range [0,1]
sigma_sample = pyDOE.lhs(n=ndim, samples=nsims, criterion='centermaximin')

smat = np.zeros((z_sample.shape[0], nsims))
print smat.shape
strans = sampleTrans2d() # instantiate class for sampling transformation
for ii,sig in enumerate(sigma_sample):
    smat[:,ii] = np.outer(gauss_trunc(z1m,strans.unit2range(sig[0])), 
                          gauss_trunc(z2m,strans.unit2range(sig[1]))).flatten() 

In [ ]:
datComp = dataCompress.dataCompress(dataMat = np.log10(smat), 
                                    histBins = None,
                                    simDesign = strans.unit2range(sigma_sample), tol=1e-5)

datComp.unitTrans()
datComp.basisCompute()

In [ ]:
datComp.pca_basis.shape

In [ ]:
## NOTE: This is the correct ordering of axes in the flattening scheme.

print strans.unit2range(sigma_sample[23])
test = 10.0**datComp.rotate2full(datComp.pca_weights[:,23]).reshape((z1m.shape[0], z2m.shape[0]))
test = np.abs(test)

plt.imshow(test,
           origin='lower', aspect='auto',
           cmap='viridis_r', extent=[z2edges[0], z2edges[-1], 
                                     z1edges[0], z1edges[-1]])
plt.xlabel('$z_2$')
plt.ylabel('$z_1$')
cb = plt.colorbar()
cb.set_label("PDF")

In [ ]:
match_compare = []
match_compare.append(datComp.match())

In [ ]:
plt.plot(1.0-match_compare[0], alpha=0.6,)
plt.minorticks_on()
plt.tick_params(which='both',direction='in',tick2On=True)
plt.xlabel('Training simulation')
plt.ylabel(r'$1-$Match')
plt.yscale('log')

### PCA GP interpolation

In [ ]:
# Instanciate a list of GP kernels and models [one for each distribution basis function]

gp_george = [None for _ in range(datComp.dim)]
k = []

for ii in range(datComp.dim):
    gp_george[ii] = gproc.gp(x = strans.range2unit(datComp.simDesign), 
                             y = datComp.pca_weights[ii,:], yerr = 1e-10,
                             p0 = np.log10(3.0*np.ones(datComp.simDesign.shape[1]+1)),
                             pmin = -10.0*np.ones(datComp.simDesign.shape[1]+1),
                             pmax = 10.0*np.ones(datComp.simDesign.shape[1]+1))

In [ ]:
# Sample the posterior distribution of the kernel parameters 
# to find MAP value for each basis function

sampler = [None for _ in range(datComp.dim)]
for ii in range(datComp.dim):
    # Set up the sampler.
    nwalkers, ndim = 36, datComp.simDesign.shape[1]+1
    sampler[ii] = emcee.EnsembleSampler(nwalkers, ndim, gp_george[ii].lnprob)

    # Initialize the walkers.
    p0 = [gp_george[ii].p0 + 1e-1 * np.random.randn(ndim)
          for i in range(nwalkers)]

    print ii, "Running burn-in"
    p0, lnp, _ = sampler[ii].run_mcmc(p0, 500)
    sampler[ii].reset()

    print ii, "Running second burn-in"
    p = p0[np.argmax(lnp)]
    p0 = [p + 1e-1 * np.random.randn(ndim) for i in xrange(nwalkers)]
    p0, _, _ = sampler[ii].run_mcmc(p0, 500)
    sampler[ii].reset()

    print ii, "Running production", '\n'
    p0, _, _ = sampler[ii].run_mcmc(p0, 1000)

In [ ]:
## Populate the GP class with the details of the kernel 
## MAP values for each frequency.
for ii in range(datComp.dim):
    gp_george[ii].emcee_kernel_map = sampler[ii].flatchain[np.argmax(sampler[ii].flatlnprobability)] 
    gp_george[ii].emcee_flatchain = sampler[ii].flatchain 
    print ii, gp_george[ii].emcee_kernel_map

In [ ]:
gpnew = []
for ii in range(datComp.dim):
    # Let's grab the MAP values
    mapparams = 10.0**sampler[ii].flatchain[np.argmax(sampler[ii].flatlnprobability)]  
    # Set up a GP kernel with the MAP values found from the sampling.
    k = mapparams[0] * kernels.ExpSquaredKernel(mapparams[1:],ndim=len(mapparams[1:]))
    # Instanciate the GP.
    gpnew.append(george.GP(k))
    # Pre-compute the factorization of the matrix.
    gpnew[ii].compute(strans.range2unit(datComp.simDesign), 1e-10)

In [ ]:
# Reconstruct parameter distribution from GP interpolants

def construct_pdf(coord, datComp, strans, gp):
    pdf = 10.0**datComp.rotate2full(np.array([gp[jj].predict(datComp.pca_weights[jj,:],
                                                       strans.range2unit(np.atleast_2d(coord)))[0][0]
                                               for jj in range(len(gp))]))
    pdf = np.abs(pdf)
    pdf = pdf - np.min(pdf)
    #pdf /= np.sum(pdf * np.outer(np.diff(z1), np.diff(z2)).flatten())
    return pdf

In [ ]:
plt.imshow(construct_pdf(np.array([0.45,0.45]), datComp, strans, gpnew).reshape((z1m.shape[0], z2m.shape[0])),
           origin='lower', aspect='auto',
           cmap='viridis_r', extent=[z2edges[0], z2edges[-1], 
                                     z1edges[0], z1edges[-1]])
cb = plt.colorbar()
cb.set_label("PDF")

In [ ]:
# Plot the analytic parameter distribution

test_against = np.outer(gauss_trunc(z1m,0.45), 
                        gauss_trunc(z2m,0.45)).flatten()

plt.imshow(test_against.reshape((z1m.shape[0], z2m.shape[0])),
           origin='lower', aspect='auto',
           cmap='viridis_r', extent=[z2edges[0], z2edges[-1], 
                                     z1edges[0], z1edges[-1]])
cb = plt.colorbar()
cb.set_label("PDF")

### MCMC on Test Population

In [ ]:
# Load in a test population

pop = np.load('./test_pop.npy')

In [ ]:
# Define a simple model

simple1d_model = model.model(data=pop, x=[z1edges,z2edges], interp=gpnew, sampTrans=strans, dataComp=datComp, 
                             pmin=np.log10(np.array([0.1,0.1])), pmax=np.log10(np.array([10.,10.])),
                             yerr=None,
                             interpType='gp1d', interpScale='log10',
                             interpErrors=True, interpHyperErrors=True, 
                             catalogType='median',analytic=None)

In [ ]:
# Filling in GP interpolant hyperparameter properties
simple1d_model.gp_kernel_map = [g.emcee_kernel_map for g in gp_george]
simple1d_model.gp_kernel_posterior = [g.emcee_flatchain for g in gp_george]

In [ ]:
# Use emcee to sample the hyper-parameter posterior distribution

nwalkers, ndim = 36, 2
sampler_simple = emcee.EnsembleSampler(nwalkers, ndim, simple1d_model.lnprob)

# Initialize the walkers.
p0 = [np.log10([3.0,3.0]) + 1e-2 * np.random.randn(ndim)
      for i in range(nwalkers)]

print "Running burn-in"
p0, lnp, _ = sampler_simple.run_mcmc(p0, 500)
sampler_simple.reset()

print "Running second burn-in"
p = p0[np.argmax(lnp)]
p0 = [p + 1e-2 * np.random.randn(ndim) for i in xrange(nwalkers)]
p0, _, _ = sampler_simple.run_mcmc(p0, 500)
sampler_simple.reset()

print "Running production"
p0, _, _ = sampler_simple.run_mcmc(p0, 5000)

In [ ]:
plt.plot(10.0**sampler_simple.flatchain[:,1])

In [ ]:
# Define the analytic model

analytic_model = model.model(data=pop, x=[z1edges,z2edges], interp=gpnew, sampTrans=strans, dataComp=datComp, 
                             pmin=np.log10(np.array([0.1,0.1])), pmax=np.log10(np.array([10.,10.])),
                             interpType='gp1d', interpScale='log10',
                             interpErrors=False, interpHyperErrors=False, 
                             catalogType='median',  
                             yerr=None, analytic=gauss_fullprod)

In [ ]:
# Use emcee to sample the hyper-parameter posterior distribution 
# (in the analytic model)

nwalkers, ndim = 36, 2
sampler_analytic = emcee.EnsembleSampler(nwalkers, ndim, analytic_model.lnprob)

# Initialize the walkers.
p0 = [np.log10([3.0,3.0]) + 1e-2 * np.random.randn(ndim)
      for i in range(nwalkers)]

print "Running burn-in"
p0, lnp, _ = sampler_analytic.run_mcmc(p0, 500)
sampler_analytic.reset()

print "Running second burn-in"
p = p0[np.argmax(lnp)]
p0 = [p + 1e-2 * np.random.randn(ndim) for i in xrange(nwalkers)]
p0, _, _ = sampler_analytic.run_mcmc(p0, 500)
sampler_analytic.reset()

print "Running production"
p0, _, _ = sampler_analytic.run_mcmc(p0, 1000)

In [ ]:
# Compare the analytic model with the GP-emulator model

c = ChainConsumer()
c.add_chain(10.0**sampler_simple.flatchain, 
            parameters=[r'$\sigma_1$', r'$\sigma_2$'], 
            name='GP model')
c.add_chain(10.0**sampler_analytic.flatchain, 
            parameters=[r'$\sigma_1$', r'$\sigma_2$'], 
            name='Analytic model')
c.configure(sigma2d=False, spacing=0., shade_alpha=0.35, 
            colors=['#1f77b4', '#d62728'], legend_kwargs=dict(prop={'size':8}))
c.configure_truth(linestyle='dashed',color='w',alpha=1.0,linewidth=1.3)
fig = c.plotter.plot(figsize="column", truth=[0.45,0.45])
fig.set_size_inches(3.0,3.0)  
plt.show()